# MVTS_Transformer 모델 실행

**!!!! inference 코드는 아래에 있습니다 !!!!**

안내사항
- checkpoint가 있는 경우, pretrain과 fine-tuning을 건너뛸 수 있습니다.  
(pretrain과 fine-tuning이 필요한 경우 코드를 일부 수정해야 합니다. 아래에 가이드라인을 작성해두었습니다.)

In [ ]:
%pip install -r failsafe_requirements.txt

## pretrain (해당 과정은 checkpoint가 있으면 건너뛸 수 있습니다.)

### 선행작업

전처리된 데이터가 `./data`에 있는지 확인해야 합니다.
- `train_val_concat.csv`: train + val user data
- `train_val_y_concat.csv`: train + val regression label file (not used, but for model compatibility)

**!! 중요 !! 코드를 수정해야 합니다.**

`./src/datasets/data.py`의 `TSRegression.load_all` 의 데이터 불러오는 내용이 아래와 같이 되어있는지 확인합니다.

```
...
# pretrain
all_df, labels_df = pd.read_csv(os.path.join(root_dir, "train_val_concat.csv"), index_col=0), \
                        pd.read_csv(os.path.join(root_dir, "train_val_y_concat.csv"), index_col=0)

# # fine-tuning (regression)
# all_df, labels_df = pd.read_csv(os.path.join(root_dir, "train_all.csv"), index_col=0), \
#                     pd.read_csv(os.path.join(root_dir, "train_y_all.csv"), index_col=0)

# # inference
# all_df, labels_df = pd.read_csv(os.path.join(root_dir, "test_all.csv"), index_col=0), \
#                     pd.read_csv(os.path.join(root_dir, "test_y_all.csv"), index_col=0)
...
```

위에 있는 pretrain 관련 코드만 활성화하고, 나머지는 주석 처리되어야 합니다.

In [ ]:
!python src/main.py \
    --output_dir output \
    --comment "pretraining through imputation" \
    --name pretrained \
    --records_file Imputation_records.xls \
    --data_dir ./data \
    --data_class etri \
    --pattern TRAIN \
    --val_ratio 0.2 \
    --epochs 2000 \
    --lr 0.001 \
    --optimizer RAdam \
    --batch_size 128 \
    --pos_encoding learnable \
    --d_model 128 \
    --seed 42

## fine-tuning (해당 과정은 checkpoint가 있으면 건너뛸 수 있습니다.)

### 선행작업

전처리된 데이터가 `./data`에 있는지 확인해야 합니다.
- `train_all.csv`: train user data
- `train_y_all.csv`: train regression label file

**!! 중요 !! 코드를 수정해야 합니다.**

`./src/datasets/data.py`의 `TSRegression.load_all` 의 데이터 불러오는 내용이 아래와 같이 되어있는지 확인합니다.

```
...
# # pretrain
# all_df, labels_df = pd.read_csv(os.path.join(root_dir, "train_val_concat.csv"), index_col=0), \
#                         pd.read_csv(os.path.join(root_dir, "train_val_y_concat.csv"), index_col=0)

# fine-tuning (regression)
all_df, labels_df = pd.read_csv(os.path.join(root_dir, "train_all.csv"), index_col=0), \
                    pd.read_csv(os.path.join(root_dir, "train_y_all.csv"), index_col=0)

# # inference
# all_df, labels_df = pd.read_csv(os.path.join(root_dir, "test_all.csv"), index_col=0), \
#                     pd.read_csv(os.path.join(root_dir, "test_y_all.csv"), index_col=0)
...
```

위에 있는 fine-tuning 코드만 활성화 되어있고, 나머지는 주석 처리되어야 합니다.

In [ ]:
!python src/main.py \
    --output_dir output \
    --comment "finetuning for regression" \
    --name finetuned_all \
    --records_file Regression_records.xls \
    --data_dir ./data \
    --data_class etri \
    --pattern TRAIN \
    --val_pattern TEST \
    --epochs 400 \
    --lr 0.1 \
    --optimizer RAdam \
    --pos_encoding learnable \
    --d_model 128 \
    --load_model ./output/pretrained_2024-06-22_04-15-36_ump/checkpoints/model_best.pth \
    --task regression \
    --change_output \
    --batch_size 128 \
    --seed 42

## inference

### 선행작업

전처리된 데이터가 `./data`에 있는지 확인해야 합니다.
- `test_all.csv`: test user data
- `test_y_all.csv`: test label file (not used, but for model compatibility)

**!! 중요 !! 코드가 올바르게 되어 있는지 확인합니다.**

`./src/datasets/data.py`의 `TSRegression.load_all` 의 데이터 불러오는 내용이 아래와 같이 되어있는지 확인합니다.
(Pretrain과 Fine-tuning을 따로 수행하지 않고 바로 inference를 수행하면, 코드를 수정하지 않아도 됩니다.(기본값))

```
...
# # pretrain
# all_df, labels_df = pd.read_csv(os.path.join(root_dir, "train_val_concat.csv"), index_col=0), \
#                         pd.read_csv(os.path.join(root_dir, "train_val_y_concat.csv"), index_col=0)

# # fine-tuning (regression)
# all_df, labels_df = pd.read_csv(os.path.join(root_dir, "train_all.csv"), index_col=0), \
#                     pd.read_csv(os.path.join(root_dir, "train_y_all.csv"), index_col=0)

# inference
all_df, labels_df = pd.read_csv(os.path.join(root_dir, "test_all.csv"), index_col=0), \
                    pd.read_csv(os.path.join(root_dir, "test_y_all.csv"), index_col=0)
...
```

위처럼 inference 관련된 데이터 부분만 주석이 해제되어 있어야 합니다.

In [4]:
!python src/main.py \
    --output_dir output \
    --comment "inference" \
    --name regression_all \
    --records_file Regression_records.xls \
    --data_dir ./data \
    --data_class etri \
    --epochs 2000 \
    --lr 0.001 \
    --optimizer RAdam \
    --pos_encoding learnable \
    --d_model 128 \
    --load_model "./output/finetuned_s1_2024-06-25_16-20-25_8yv/checkpoints/model_best.pth" \
    --batch_size 128 \
    --task regression \
    --inference_mode \
    --seed 42

2024-06-28 20:50:48,703 | INFO : Loading packages ...
2024-06-28 20:50:49,963 | INFO : Stored configuration file in 'output/regression_all_2024-06-28_20-50-49_jeA'
2024-06-28 20:50:49,964 | INFO : Running:
src/main.py --output_dir output --comment inference --name regression_all --records_file Regression_records.xls --data_dir ./data --data_class etri --epochs 2000 --lr 0.001 --optimizer RAdam --pos_encoding learnable --d_model 128 --load_model ./output/finetuned_s1_2024-06-25_16-20-25_8yv/checkpoints/model_best.pth --batch_size 128 --task regression --inference_mode --seed 42

2024-06-28 20:50:49,997 | INFO : Using device: cuda
Sorted IDs: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  8

inference 이후 결과를 전처리합니다.

In [15]:
import numpy as np
import pandas as pd

In [16]:
test_csv = pd.read_csv("../../data/answer_sample.csv")
reg_result = np.load("sorted_res_reg.npy")

reg_result_pd = pd.DataFrame(reg_result, columns=["y1", "y2", "y3", "y4", "y5", "y6", "y7"])
concated_csv = pd.concat([test_csv, reg_result_pd], axis=1)

In [17]:
# Calculate regression
concated_csv["Q1_"] = concated_csv["y1"]
concated_csv["Q2_"] = concated_csv["y2"]
concated_csv["Q3_"] = concated_csv["y3"]
concated_csv["S1_"] = concated_csv["y4"]
concated_csv["S2_"] = concated_csv["y4"] / (concated_csv["y4"] + concated_csv["y5"]) * 100
concated_csv["S3_"] = concated_csv["y6"]
concated_csv["S4_"] = concated_csv["y5"] - concated_csv["y6"] - concated_csv["y7"]

# Q2와 Q3 칼럼을 변환 (각각의 값이 그룹 평균을 초과하는지 여부)
concated_csv.loc[:, 'Q1'] = (concated_csv['Q1_'] > concated_csv.groupby('subject_id')['Q1_'].transform('mean')).astype(int)
concated_csv.loc[:, 'Q2'] = (concated_csv['Q2_'] > concated_csv.groupby('subject_id')['Q2_'].transform('mean')).astype(int)
concated_csv.loc[:, 'Q3'] = (concated_csv['Q3_'] > concated_csv.groupby('subject_id')['Q3_'].transform('mean')).astype(int)

# S1, S2, S3, S4 칼럼을 변환 (특정 조건에 따라 0 또는 1로 설정)
concated_csv.loc[:, 'S1'] = ((concated_csv['S1_'] >= 7*60*60) & (concated_csv['S1_'] <= 9*60*60)).astype(int)
concated_csv.loc[:, 'S2'] = (concated_csv['S2_'] >= 0.85*100).astype(int)
concated_csv.loc[:, 'S3'] = (concated_csv['S3_'] <= 30*60).astype(int)
concated_csv.loc[:, 'S4'] = (concated_csv['S4_'] <= 20*60).astype(int)

# 칼럼 타입을 정수형으로 변환
concated_csv[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3', 'S4']] = concated_csv[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3', 'S4']].astype(int)
concated_csv = concated_csv[["subject_id", "date", "Q1", "Q2", "Q3", "S1", "S2", "S3", "S4"]]

In [20]:
concated_csv.to_csv("../../predictions/tst_result_q_column.csv", index=False); concated_csv

,subject_id,date,Q1,Q2,Q3,S1,S2,S3,S4
0,5,2023-11-05,1,1,1,0,0,1,1
1,5,2023-11-06,1,1,1,0,0,1,1
2,5,2023-11-07,0,1,0,0,0,1,1
3,5,2023-11-08,1,0,0,0,0,1,1
4,5,2023-11-09,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...
110,8,2023-11-05,1,0,0,0,0,1,1
111,8,2023-11-06,0,1,0,0,0,1,1
112,8,2023-11-07,0,1,1,0,0,1,1
113,8,2023-11-08,0,1,1,0,0,1,1
